In [2]:
#Installation des dépendances en amont si non-présentes dans l'environnement
!pip install numpy pandas scikit-learn tensorflow imblearn openpyxl

In [3]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy
import pandas as pd

In [4]:
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()

In [5]:
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)
data_dict.keys()

dict_keys(['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players', 'clubs_link_api'])

In [6]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [7]:
df_games = data_dict["games"]
df_games.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type'],
      dtype='object')

In [8]:
# Fonction pour déterminer le résultat
def determine_result(row):
    if row['home_club_goals'] > row['away_club_goals']:
        return 1
    elif row['home_club_goals'] < row['away_club_goals']:
        return -1
    else:
        return 0

# Ajout de la colonne 'resultat'
df_games['resultat'] = df_games.apply(determine_result, axis=1)
df_games.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type', 'resultat'],
      dtype='object')

In [44]:
#Toutes les fonctions
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

def last_five_games(df_games, club_id, date, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié avant une date donnée en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date (str ou pd.Timestamp) : Date avant laquelle les matchs doivent être récupérés.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Assurer que la colonne 'date' est au format datetime
    df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrage des matchs avant la date donnée
    matchs_du_club = df_games[df_games['date'] < date]
    
    if lieu == 'domicile':
        matchs_du_club = matchs_du_club[matchs_du_club['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = matchs_du_club[matchs_du_club['away_club_id'] == club_id]
    else:  # 'tous'
        matchs_du_club = matchs_du_club[(matchs_du_club['home_club_id'] == club_id) | (matchs_du_club['away_club_id'] == club_id)]
    
    # Tri des matchs par date et sélection des 5 derniers
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]
    
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def starting_lineup_match(df_game_lineups, match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    #df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)




In [109]:
df_games.head()
df_games['date'] = pd.to_datetime(df_games['date'])
df_games = df_games.sort_values(by='date', ascending=True)

In [110]:
# Supposons que df_games est votre DataFrame original
selected_columns = ["game_id", "season", "date", "home_club_id", "away_club_id", "home_club_position", "away_club_position", "resultat"]
# Création du nouveau DataFrame avec seulement les colonnes sélectionnées
df_final = df_games[selected_columns].dropna()
df_final.head(5)

,game_id,season,date,home_club_id,away_club_id,home_club_position,away_club_position,resultat
46104,2224728,2012,2012-07-13,678,1053,6.0,7.0,0
10656,2222962,2012,2012-07-13,4482,2477,9.0,8.0,0
53514,2222961,2012,2012-07-14,338,2783,6.0,12.0,1
12,2222958,2012,2012-07-14,6414,6993,3.0,13.0,1
62,2224732,2012,2012-07-14,5817,5724,1.0,12.0,1


In [111]:
df_players = data_dict['players']  # Assurez-vous d'avoir le DataFrame correct pour les évaluations des joueurs
df_game_lineups = data_dict['game_lineups']

In [112]:
# Ajout de la valeur marchande totale des équipes
df_final['home_team_market_value'] = df_final.apply(lambda row: sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, row['home_club_id']), row['home_club_id'])), axis=1)
df_final['away_team_market_value'] = df_final.apply(lambda row: sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, row['away_club_id']), row['away_club_id'])), axis=1)
df_final.head()

,game_id,season,date,home_club_id,away_club_id,home_club_position,away_club_position,resultat,home_team_market_value,away_team_market_value
46104,2224728,2012,2012-07-13,678,1053,6.0,7.0,0,8250000.0,4700000.0
10656,2222962,2012,2012-07-13,4482,2477,9.0,8.0,0,0.0,0.0
53514,2222961,2012,2012-07-14,338,2783,6.0,12.0,1,36000000.0,0.0
12,2222958,2012,2012-07-14,6414,6993,3.0,13.0,1,0.0,0.0
62,2224732,2012,2012-07-14,5817,5724,1.0,12.0,1,2275000.0,8100000.0


In [113]:
def calculer_forme(df_games, club_id, date_du_match, lieu='tous'):
    """
    Calcule la forme d'un club sur ses 5 derniers matchs avant la date du match spécifiée.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date_du_match (str ou pd.Timestamp) : Date du match pour lequel la forme est calculée.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    int : Les points accumulés par le club sur ses 5 derniers matchs.
    """
    # Assurer que la colonne 'date' est au format datetime
    #df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrer les matchs précédents et trier par date
    matchs_precedents = last_five_games(df_games, club_id, date_du_match, lieu)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points


In [114]:
# Ajout de la forme des équipes
df_final['home_team_form'] = df_final.apply(lambda x: calculer_forme(df_games, x["home_club_id"], x["date"], "domicile"), axis=1)
df_final['away_team_form'] = df_final.apply(lambda x: calculer_forme(df_games, x["away_club_id"],  x["date"], "exterieur"), axis=1)
df_final.head()

,game_id,season,date,home_club_id,away_club_id,home_club_position,away_club_position,resultat,home_team_market_value,away_team_market_value,home_team_form,away_team_form
46104,2224728,2012,2012-07-13,678,1053,6.0,7.0,0,8250000.0,4700000.0,0,0
10656,2222962,2012,2012-07-13,4482,2477,9.0,8.0,0,0.0,0.0,0,0
53514,2222961,2012,2012-07-14,338,2783,6.0,12.0,1,36000000.0,0.0,0,0
12,2222958,2012,2012-07-14,6414,6993,3.0,13.0,1,0.0,0.0,0,0
62,2224732,2012,2012-07-14,5817,5724,1.0,12.0,1,2275000.0,8100000.0,0,0


In [245]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())

Taille du df 33640
Nombre de lignes avec 0 dans 'resultat': 8349
Nombre de lignes avec -1 dans 'resultat': 10326
Nombre de lignes avec 1 dans 'resultat': 14965
PB home market value: 0
PB away market value: 0
PB home market value: 0
PB away market value: 0


In [246]:
# Suppression des lignes où 'home_team_market_value' < 1
df_to_train = df_to_train.loc[df_final['home_team_market_value'] >= 1]
# Suppression des lignes où 'away_team_market_value' < 1
df_to_train = df_to_train.loc[df_final['away_team_market_value'] >= 1]
# Suppression des lignes où 'home_team_form' < 1
df_to_train = df_to_train.loc[df_final['home_team_form'] >= 1]
# Suppression des lignes où 'away_team_form' < 1
df_to_train = df_to_train.loc[df_final['away_team_form'] >= 1]
df_to_train.head()

,home_club_id,away_club_id,resultat,home_team_market_value,away_team_market_value,home_team_form,away_team_form
21541,190,1053,1,46350000.0,4700000.0,3,1
42256,660,3592,1,79000000.0,6650000.0,3,3
38517,3592,338,-1,6650000.0,36000000.0,3,3
6925,1053,2778,0,4700000.0,10950000.0,3,3
6640,16704,932,1,70200000.0,46800000.0,3,3


In [247]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils import to_categorical

# Supposons que df_final est déjà défini et contient les données nécessaires

# 1. Préparation et nettoyage des données
conditions = (
    (df_final['home_team_market_value'] >= 1) & 
    (df_final['away_team_market_value'] >= 1) & 
    (df_final['home_team_form'] >= 1) & 
    (df_final['away_team_form'] >= 1)
)
df_to_train = df_final[conditions]
df_to_train = df_to_train.drop(["season", "game_id", "away_club_position", "home_club_position", "date"], axis=1).dropna()

#import openpyxl
#df_final.to_excel("df_final.xlsx", index=False)
#df_to_train.to_excel("df_to_train.xlsx", index=False)

df_to_train.head(10)

,home_club_id,away_club_id,resultat,home_team_market_value,away_team_market_value,home_team_form,away_team_form
21541,190,1053,1,46350000.0,4700000.0,3,1
42256,660,3592,1,79000000.0,6650000.0,3,3
38517,3592,338,-1,6650000.0,36000000.0,3,3
6925,1053,2778,0,4700000.0,10950000.0,3,3
6640,16704,932,1,70200000.0,46800000.0,3,3
31071,865,5724,1,23700000.0,8100000.0,3,3
18116,5817,2414,-1,2275000.0,2350000.0,4,3
38518,10690,3592,-1,8000000.0,6650000.0,3,3
39101,2778,1177,1,10950000.0,9900000.0,1,3
31033,964,232,1,81400000.0,47000000.0,6,6


In [248]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())

Taille du df 33640
Nombre de lignes avec 0 dans 'resultat': 8349
Nombre de lignes avec -1 dans 'resultat': 10326
Nombre de lignes avec 1 dans 'resultat': 14965
PB home market value: 0
PB away market value: 0
PB home market value: 0
PB away market value: 0


"\nfrom keras.layers import LSTM\n\nmodel = Sequential()\nmodel.add(LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))\nmodel.add(Dense(3, activation='softmax'))\n"

Epoch 1/50
673/673 [==============================] - 3s 3ms/step - loss: 1.0550 - accuracy: 0.4866 - val_loss: 1.0265 - val_accuracy: 0.4945
Epoch 2/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0425 - accuracy: 0.4919 - val_loss: 1.0438 - val_accuracy: 0.4769
Epoch 3/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0378 - accuracy: 0.4964 - val_loss: 1.0454 - val_accuracy: 0.4793
Epoch 4/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0375 - accuracy: 0.4978 - val_loss: 1.0226 - val_accuracy: 0.4958
Epoch 5/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0362 - accuracy: 0.4977 - val_loss: 1.0187 - val_accuracy: 0.4992
Epoch 6/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0358 - accuracy: 0.4996 - val_loss: 1.0422 - val_accuracy: 0.4940
Epoch 7/50
673/673 [==============================] - 2s 3ms/step - loss: 1.0360 - accuracy: 0.4957 - val_loss: 1.0260 - val_accuracy: 0.4932
Epoch 

1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Olympique Lyonnais et Paris Saint-Germain Football Club : victoire
1/1 [==============================] - 0s 28ms/step
Résultat prédit pour le match entre les clubs Olympique Lyonnais et Clermont Foot 63 : nul
1/1 [==============================] - 0s 36ms/step
Résultat prédit pour le match entre les clubs Paris Saint-Germain Football Club et Clermont Foot 63 : nul
1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Olympique Lyonnais et Association sportive de Monaco Football Club : défaite
1/1 [==============================] - 0s 33ms/step
Résultat prédit pour le match entre les clubs Association sportive de Monaco Football Club et Olympique Lyonnais : nul
1/1 [==============================] - 0s 28ms/step
Résultat prédit pour le match entre les clubs Real Madrid Club de Fútbol et Unión Deportiva Almería S.A.D. : nul
1/1 [==================